# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [31]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'C:/Users/USER/Desktop/Github/ML100Days/homework/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [34]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
#票價(Fare)對艙(Cabin)做群聚編碼:
df['Cabin'] = df['Cabin'].fillna('None')

mean_df = df.groupby(['Cabin'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Cabin'])['Fare'].median().reset_index()
max_df = df.groupby(['Cabin'])['Fare'].max().reset_index()
#min_df = df.groupby(['Cabin'])['Fare'].min().reset_index()
#count_df = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.value_counts()).reset_index()


temp = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp = pd.merge(temp, median_df, how='left', on=['Cabin'])
temp = pd.merge(temp, max_df, how='left', on=['Cabin'])
#temp = pd.merge(temp, min_df, how='left', on=['Cabin'])
#temp = pd.merge(temp, count_df, how='left', on=['Cabin'])

temp.columns = ['Cabin',
                'Fare_Mean',
                'Fare_Mode',
                'Fare_Median',
                'Fare_Max'
                #'Fare_min',
                #'Fare_count'
               ]
temp

,Cabin,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,A10,40.125000,40.1250,40.12500,40.1250
1,A14,52.000000,52.0000,52.00000,52.0000
2,A16,39.600000,39.6000,39.60000,39.6000
3,A19,26.000000,26.0000,26.00000,26.0000
4,A20,56.929200,56.9292,56.92920,56.9292
...,...,...,...,...,...
143,F38,7.750000,7.7500,7.75000,7.7500
144,F4,39.000000,39.0000,39.00000,39.0000
145,G6,13.581250,10.4625,13.58125,16.7000
146,None,19.157325,8.0500,10.50000,512.3292


In [35]:
df= pd.merge(df, temp, how = 'left', on = ['Cabin'])
df = df.drop(['Cabin'], axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.0500,10.5000,512.3292
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.0500,10.5000,512.3292
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.0500,10.5000,512.3292


In [36]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [37]:
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Fare_Mean','Fare_Mode','Fare_Median','Fare_Max'] , axis=1)

# 原始特徵 + 羅吉斯迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [38]:
# 新特徵 + 邏輯斯迴歸

train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6892504646627564

反而變差......
具理解，應該是因為"Cabin類別型特徵搭配Fare數值型特徵"的組合不具特徵重要性，因此模型表現未能提升